Nama : Shopi Nurhidayanti
---
Institut Teknologi Garut
---
Teknik Informatika // Project Akhir Information Retrieval
---





# Import dataset

link dataset yang digunakan : https://github.com/ridife/dataset-idsa - dataset hasil crawling yang mencakup keseluruhan

In [23]:
from google.colab import files
files.upload()

Saving dataset_tweet_sentiment_opini_film.csv to dataset_tweet_sentiment_opini_film.csv


{'dataset_tweet_sentiment_opini_film.csv': b'\xef\xbb\xbfId,Sentiment,Text Tweet\r\n1,negative,Jelek filmnya... apalagi si ernest gak mutu bgt actingnya... film sampah\r\n2,negative,Film king Arthur ini film paling jelek dari seluruh cerita King Arthur \r\n3,negative,@beexkuanlin Sepanjang film gwa berkata kasar terus pada bapaknya\r\n4,negative,Ane ga suka fast and furious..menurutku kok jelek ya tu film\r\n5,negative,"@baekhyun36 kan gua ga tau film nya, lu bilang perang perangan/? Perang""an disebut ama rp yaoi jadi ambigu :v"\r\n6,negative,tolong editingnya yg bagus ya. Saya sering kecewa dgn film indonesia. Ditunggu filmnya!!\r\n7,negative,"Kecewa dgn salah satu aktornya yg ternyata pendukung penista agama. Ah, saya harus bersabar utk tak menonton film ini."\r\n8,negative,Kecewa parah sama film the guys. Dear @radityadika sorry to say this.\r\n9,negative,Banyak yg kecewa abis nonton film ini :(\r\n10,negative,#TheMummy 2017 adalah film yang paling memgecewakan saya selama hidup. Y

# Import library

In [1]:
import pandas as pd 
import numpy as np 

data = pd.read_csv('dataset_tweet_sentiment_opini_film.csv', index_col = 0)
data 

,Sentiment,Text Tweet
Id,,
1,negative,Jelek filmnya... apalagi si ernest gak mutu bg...
2,negative,Film king Arthur ini film paling jelek dari se...
3,negative,@beexkuanlin Sepanjang film gwa berkata kasar ...
4,negative,Ane ga suka fast and furious..menurutku kok je...
5,negative,"@baekhyun36 kan gua ga tau film nya, lu bilang..."
...,...,...
196,positive,Fargo juga adaptasi dari film yang cukup berha...
197,positive,637.000 waw ini sangat keren flm horor dng jum...
198,positive,@filmziarah film yang tenang dan menghanyutkan...


In [2]:
data.info() # melihat informasi dari dataset 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentiment   200 non-null    object
 1   Text Tweet  200 non-null    object
dtypes: object(2)
memory usage: 4.7+ KB


In [3]:
data.shape # mengetahui panjang dimensi dari data

(200, 2)

In [4]:
data.describe() # deskripsi statistik pada data

,Sentiment,Text Tweet
count,200,200
unique,2,198
top,negative,nah ini. coba nonton filmnya deh. asik parah
freq,100,2


In [5]:
data.isnull().sum() # mengecek missing value pada data

Sentiment     0
Text Tweet    0
dtype: int64

# Data transformation

In [6]:
x = data['Text Tweet']
y = data['Sentiment']

In [7]:
from tokenizer_input import CustomTokenizerExample

In [8]:
token = CustomTokenizerExample()
token.text_data_cleaning("hari ini adalah hari yang cerah")

['hari', 'ini', 'adalah', 'hari', 'yang', 'cerah']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=token.text_data_cleaning)


In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=data.Sentiment, random_state=0)

In [11]:
x_train.shape, x_test.shape

((160,), (40,))

In [12]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [13]:
clasifer = LinearSVC()
pipeline = Pipeline([('tdidf', tfidf), ('clf', clasifer)])

In [14]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('tdidf',
                 TfidfVectorizer(tokenizer=<bound method CustomTokenizerExample.text_data_cleaning of <tokenizer_input.CustomTokenizerExample object at 0x0000020261DC2B80>>)),
                ('clf', LinearSVC())])

# Cek Modeling

In [15]:
y_pred = pipeline.predict(x_test)

In [16]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

confusion_matrix(y_test, y_pred)

array([[16,  4],
       [ 3, 17]], dtype=int64)

In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.84      0.80      0.82        20
    positive       0.81      0.85      0.83        20

    accuracy                           0.82        40
   macro avg       0.83      0.82      0.82        40
weighted avg       0.83      0.82      0.82        40



# Model Serialization

In [18]:
import joblib 
joblib.dump(pipeline, 'sentimen-opini-film.pkl')
j

['sentimen-opini-film.pkl']

# Predict sentimen using model

In [19]:
prediction = pipeline.predict(['film ini bagus'])
if prediction =='positive':
    print('Result : review positif')
else: 
    print('Result : review negatif')     

Result : review positif


In [20]:
new_review = []
pred_sentiment = []

while True:
    review = input('silahkan masukan review pada film : ')
    if review == 'skip': 
        print('sampai jumpa nanti') 
        break 
    else: 
        prediction = pipeline.predict([review])

        if prediction == 'positive': 
            result = 'Positif'
            print('Result : Hasil review positif')
        else : 
            result = 'Negatif'
            print('Result : Hasil review negatif')
    
    new_review.append(review)
    pred_sentiment.append(result)

Result : Hasil review negatif
Result : Hasil review positif
sampai jumpa nanti


In [21]:
result_summary = pd.DataFrame({
    'New Review' : new_review, 
    'Sentiment' : pred_sentiment
})

result_summary.to_csv('Predict_sentiment.csv', sep = '\t', encoding = 'UTF-8', index = False)
result_summary

,New Review,Sentiment
0,ini jelek,Negatif
1,ini bagus,Positif
